In [3]:
# Python 2/3 compatibility
from __future__ import print_function
 # Needed Libraries
import cv2
import numpy as np
import sys
import math
from numpy import unravel_index

In [2]:
# Status: need ground truth, consider 2 in one image, max detected around 22, play with margins
#HSV colors: http://colorizer.org/
#doesn't work on 0003, second one in 0007, 
#(23, 25): 0003, 0010, (0012, 0017, 0018 not checked), 25
# got 25 on red = 0, 12
imname = 'Images/Am_Rojo0025.jpg'
resizef = 2 # resizing
# Preprocessing
img0 = cv2.imread(imname) # Read image
imgl, imgw = img0.shape[:2]
imgl=int(imgl/resizef)
imgw=int(imgw/resizef)
img = cv2.resize(img0,(imgw, imgl), interpolation = cv2.INTER_CUBIC) # Resizing image
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# black color
blacked = np.array([0,0,0]) 

# red mask
lower_red = np.array([0,125,75])
upper_red = np.array([10,255,255])
maskr = cv2.inRange(hsv, lower_red, upper_red)

# yellow mask
lower_yellow = np.array([22, 150,100])
upper_yellow = np.array([25,255,255])
masky = cv2.inRange(hsv, lower_yellow, upper_yellow)

mask = maskr+masky # adding the masks

res = cv2.bitwise_and(img,img, mask= mask) # anding input with mask
grayres = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY) # Convert to gray scale

# Show images
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('Sample Outputs/res0.jpg',res) # Save edges


print('shape = ',res.shape)

NameError: name 'cv2' is not defined

In [3]:
# 
count = 0
for i in range (0, imgl):
    for j in range (0, imgw):
        if (res[i][j][2] != 0): # not black
            count+=1
print(count)

407


In [4]:
label = 2 # current new label
labels = np.zeros((imgl, imgw)) # 2D array of labels
# Put labels
equality = [[1]]
l1, l2, l3, l4 = 0, 0, 0, 0
for i in range (0, imgl):
    for j in range (0, imgw):
        if (res[i][j][2] != 0): # not black
            if(i-1>=0 and j-1>=0 and j+1<imgw): # away from edges, get neighbor labels
                l1 = labels[i-1][j-1]
                l2 = labels[i-1][j]
                l3 = labels[i-1][j+1]
                l4 = labels[i][j-1]
                flag = True
                # append to equality table
                for entry in equality:
                    if(flag):
                        for enn in entry:
                            if(flag):
                                if(int(enn) == int(l1)):
                                    flag = False
                                    labels[i][j] = entry[0] # append as the first label
                                    if(l2 != 0 and l2 != l1 and l2 not in entry): entry.append(l2)
                                    if (l3 != 0 and l3 != l1 and l3 not in entry): entry.append(l3)
                                    if (l4 != 0 and l4 != l1 and l4 not in entry): entry.append(l4)
                                elif (int(enn) == int(l2)):
                                    flag = False
                                    labels[i][j] = entry[0] # append as the first label
                                    if(l1 != 0 and l2 != l1 and l1 not in entry): entry.append(l1)
                                    if (l3 != 0 and l2 != l3 and l3 not in entry): entry.append(l3)
                                    if (l4 != 0 and l2 != l4 and l4 not in entry): entry.append(l4)
                                elif (int(enn) == int(l3)):
                                    flag = False
                                    labels[i][j] = entry[0] # append as the first label
                                    if(l2 != 0 and l2 != l3 and l2 not in entry): entry.append(l2)
                                    if (l1 != 0 and l3 != l1 and l1 not in entry): entry.append(l1)
                                    if (l4 != 0 and l3 != l4 and l4 not in entry): entry.append(l4)
                                elif (int(enn) == int(l4)):
                                    flag = False
                                    labels[i][j] = entry[0] # append as the first label
                                    if(l2 != 0 and l2 != l4 and l2 not in entry): entry.append(l2)
                                    if (l3 != 0 and l4 != l3 and l3 not in entry): entry.append(l3)
                                    if (l1 != 0 and l4 != l1 and l1 not in entry): entry.append(l1)
                
                if(flag):
                    labels[i][j] = label
                    equality.append([label])
                    label += 1
print(label)
#cv2.imshow('labels',labels)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

62


In [5]:
#print(equality)

In [6]:
# Get max count
maxi = 0
maximum = 0
cc = 0
ii = 0

for entry in equality:
    cc = 0
    flag = True
    for enn in entry:
        if(flag):
            for i in range (0, imgl):
                for j in range (0, imgw):
                    if (enn == labels[i][j]):
                        cc += 1
                        flag = False
    if (cc > maximum):
        maximum = cc
        maxi = ii       
    ii += 1

print(maxi, maximum)

19 137


In [7]:
labels2 = np.zeros((imgl, imgw)) # 2D array of labels
mini, minj = 1000, 1000
maxii, maxjj = -1, -1
for i in range (0, imgl):
    for j in range (0, imgw):
        if (labels[i][j] in equality[maxi]): # not labeled
            labels2[i][j] = 255
            if(i<mini): mini = i
            if(i>maxii): maxii = i
            if(j<minj): minj = j
            if(j>maxjj): maxjj = j  
#cv2.imshow('labels2',labels2)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
print(mini, maxii)
print(minj, maxjj)

202 230
264 295


In [8]:
img = cv2.resize(img0,(imgw, imgl), interpolation = cv2.INTER_CUBIC) # Resizing image
img = cv2.rectangle(img,(minj,mini),(maxjj,maxii),(255,0,0),2)
cv2.imwrite('Sample Outputs/' + imname,img) # Save edges
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [329]:
"""
#0001, 0002 only red, not 0003, 21
# red mask
lower_red = np.array([0,150,75])
upper_red img = cv2.resize(img0,(imgw, imgl), interpolation = cv2.INTER_CUBIC) # Resizing image
circles = cv2.HoughCircles(grayres,cv2.HOUGH_GRADIENT,1,1,param1=50,param2=30,minRadius=30,maxRadius=100)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
# draw the outer circle
    cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
# draw the center of the circle
    cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)= np.array([5,255,255])
maskr = cv2.inRange(hsv, lower_red, upper_red)

# yellow mask
lower_yellow = np.array([15, 225,100])
upper_yellow = np.array([20,255,255])

# 0025, 24, 4, 23, 22, 21, 20 (big), 19, 
# red mask
lower_red = np.array([0,125,75])
upper_red = np.array([5,255,255])
maskr = cv2.inRange(hsv, lower_red, upper_red)

# yellow mask
lower_yellow = np.array([15, 150,100])
upper_yellow = np.array([25,255,255])
masky = cv2.inRange(hsv, lower_yellow, upper_yellow)
"""


'\n#0001, 0002 only red, not 0003, 21\n# red mask\nlower_red = np.array([0,150,75])\nupper_red img = cv2.resize(img0,(imgw, imgl), interpolation = cv2.INTER_CUBIC) # Resizing image\ncircles = cv2.HoughCircles(grayres,cv2.HOUGH_GRADIENT,1,1,param1=50,param2=30,minRadius=30,maxRadius=100)\ncircles = np.uint16(np.around(circles))\nfor i in circles[0,:]:\n# draw the outer circle\n    cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)\n# draw the center of the circle\n    cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)= np.array([5,255,255])\nmaskr = cv2.inRange(hsv, lower_red, upper_red)\n\n# yellow mask\nlower_yellow = np.array([15, 225,100])\nupper_yellow = np.array([20,255,255])\n\n# 0025, 24, 4, 23, 22, 21, 20 (big), 19, \n# red mask\nlower_red = np.array([0,125,75])\nupper_red = np.array([5,255,255])\nmaskr = cv2.inRange(hsv, lower_red, upper_red)\n\n# yellow mask\nlower_yellow = np.array([15, 150,100])\nupper_yellow = np.array([25,255,255])\nmasky = cv2.inRange(hsv, lower_yellow, upper_yellow)\n